In [42]:
import pandas as pd
import numpy as np
import seaborn as sns
from langdetect import detect
from googletrans import Translator
import os
import pickle

%matplotlib

Using matplotlib backend: Qt5Agg


In [43]:
admission_data = pd.read_csv('Admission data RandID.csv')
antibiotics = pd.read_csv('Antibiotics RandID.csv')
cultures = pd.read_csv('Cultures w remarks RandID.csv').drop('Unnamed: 12', axis=1)

names = list(admission_data.columns) + list(antibiotics.columns) + list(cultures.columns)
names = [name for name in names if detect(name) == 'he']
translated = [
    'diagnosis_type',
    'status_at_reception',
    'status_before_hospitalization',
    'arrived_from',
    'reffered_by',
    'sample_taken_from',
    'autolab',
    'test_data',
    'sticker_number',
    'test_auth_time',
    'age_',
    'current_age',
    'gender_',
    'dob',
    'bacteria',
    'bacteria_original_result',
    'antibiotic',
    'antibiotic_original_result',
    'antibiotic_sensitivity']
names = dict(zip(names,translated))

cultures = cultures.rename(names, axis=1)
admission_data = admission_data.rename(names, axis=1)
antibiotics = antibiotics.rename(names, axis=1)

{print(k,v) for k,v in names.items()}
print(list(admission_data.columns) + list(antibiotics.columns) + list(cultures.columns))

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


סוג.אבחנה diagnosis_type
מצב.תפקודי.בקבלה status_at_reception
מצב.תפקויד.לפני.אשפוז status_before_hospitalization
הגיע.למחלקה.מ. arrived_from
גורם מפנה reffered_by
מקום לקיחת דגימה sample_taken_from
בדיקה-אוטולאב autolab
תאריך בדיקה test_data
מספר מדבקה sticker_number
תאריך ושעת אישור בדיקה test_auth_time
גיל age_
גיל נוכחי current_age
מין gender_
תאריך לידה dob
חיידק bacteria
תוצאת חיידק -דיווח מקורי bacteria_original_result
אנטיביוטיקה antibiotic
תוצאת אנטיביוטיקה-דיווח מקורי antibiotic_original_result
רגישות לאנטיביוטיקה antibiotic_sensitivity
['RANDID', 'Gender', 'age', 'ER_Admission_No', 'ER_Arrival_Date', 'Release_Date', 'Name', 'Diagnosis', 'Diagnosis_Date', 'Name.1', 'Icd', 'diagnosis_type', 'status_at_reception', 'status_before_hospitalization', 'arrived_from', 'grpd_diagnosis', 'RANDID', 'Admission_Date', 'Release_Date', 'Drug', 'Name', 'Way_Of_Giving', 'Dosage', 'Dosage_Unit', 'from_date', 'to_date', 'RANDID', 'reffered_by', 'sample_taken_from', 'autolab', 'test_data', 'stic

In [44]:
trans = Translator()
def parser(s):
#     try:
#         det = trans.detect(s)
#     except Exception as e:
#         print('Fail for', s, type(s), str(e))
#         return s
#     if det.confidence < 0.95 or det.lang !='iw':
#         return s
    return trans.translate(s).text

cols_translate = [
        'Gender',
        'Name',
        'diagnosis_type',	
        'status_at_reception',
        'status_before',
        'arrived_from',
        'sample_taken', 
        'autolab',
    ]    
    
def parse_dic(df):
    trans=Translator()
    filename = '/home/yairi/AntRes/dictionary.pickle' ## CHANGE THIS
    if os.path.exists(filename):
        with open(filename, 'rb') as f:
            dic = pickle.load(f)
    else:
        dic = dict()
    words = set()
    for col in cols_translate:
        if not col in df.columns:
            continue
        words = words.union(df[col].unique())

    words = list(words.difference(dic.keys()))
    words = [word for word in words if not pd.isnull(word)]
    print('Words we ask Google to translate', words)
    new_dict = dict(zip(words, [tr.text for tr in trans.translate(words)]))
    dic.update(new_dict)
    with open(filename, 'wb') as f:
        pickle.dump(dic, f)
    return dic

parse_dic(antibiotics)
parse_dic(admission_data)
#parse_dic(cultures)

Words we ask Google to translate []
Words we ask Google to translate []


{'פנימית ג ישנה - לא פעיל': 'There is also internal - Inactive',
 'מחלקת אף אוזן גרון': 'Otolaryngology',
 'פנימית ד': 'Internal D',
 'פנימית ג': 'Medicine C',
 'האגף האורולוגי-בלינסון': 'Beilinson Hospital Urology Department',
 'מחלקת עור': 'Department of Dermatology',
 'תלות בקניות': 'Dependence on shopping',
 'כירורגית ב': 'Surgery in',
 'המחלקה לאונקולוגיה': 'Department of Oncology',
 'עיניים - השרון - לא פעיל': 'Eyes - Sharon - Inactive',
 'תלות בהסעות': 'Dependence on transportation',
 'אף אוזן גרון - השרון': 'ENT - Sharon',
 'פנימית ג - השרון - לא פעיל': 'Medicine C - Sharon - Inactive',
 'נקבה': 'female',
 'כוויות': 'Burns',
 'תלות בלקיחת תרופות': 'Dependence of taking drugs',
 'כירורגית א': 'A surgical',
 'אורולוגיה - השרון': 'Urology - Sharon',
 'ניתוחי לב חזה': 'Chest heart surgery',
 'מחלקת עיניים - לא פעיל': 'Department of Ophthalmology - Inactive',
 'מחלקת עיניים - השרון': 'Department of Ophthalmology - Sharon',
 'בית חולים אחר': 'Another hospital',
 'פנימית א': 'Internal